### This is testing script to pull the most in focus slice out of images based on shannon entropy of normalized discrete cosine transform. This is similar to previous scripts but it is looking at nd2 files instead of a series of .tiffs

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%gui qt
import napari
from skimage import io
import seaborn as sns
from nd2reader import ND2Reader

In [35]:
orig_img_dir = r'Z:\TAD\200610_adipogenesis_timecourse_r10_v2\gr-rxr-set'
new_img_dri = r'Z:\TAD\200610_adipogenesis_timecourse_r10_v2\gr-rxr-set\temp_imgs'

In [36]:
os.chdir(orig_img_dir)

In [3]:
os.listdir()


['dapi0.tif',
 'dapi0_segm_.tiff',
 'dapi1.tif',
 'dapi1_segm_.tiff',
 'dapi2.tif',
 'dapi2_segm_.tiff',
 'stad424',
 'stad425_rois002.nd2',
 'stad425_rois003.nd2',
 'stad425_rois004.nd2']

In [23]:
'''This will calculate and return the shannon entropy of the normalized discrete cosine transform. This is 
taken from strategy used in doi: 10.1038/nbt.3708. takes in a 2D array and spits out a scaler, the dcst.
'''
def dcts(img_):
    dis_cos = dct(img_) #take the discrete cosine transform of the image
    l2 = np.sqrt(np.sum(np.square(img_))) #perform l2 normalization
    inner_term = np.divide(dis_cos, l2) #inner term of the shannon enropy
    '''need to get rid of any zeros that occur in inner term'''
    inner_term[inner_term == 0] = .0001
    first_term = np.abs(inner_term) #first term of shannon entropy
    second_term = np.log2(np.abs(inner_term)) #second term of shannon entropy 
    return(np.multiply(-1, np.sum(np.multiply(first_term, second_term))))

In [4]:
img_filename_lst = ['stad425_rois002.nd2',
 'stad425_rois003.nd2',
 'stad425_rois004.nd2']

In [26]:
roi2_multipoints = ND2Reader('stad425_rois002.nd2')
roi4_multipoints = ND2Reader('stad425_rois004.nd2')

In [27]:
roi2_multipoints.sizes

{'x': 2048, 'y': 2044, 'c': 4, 't': 1, 'z': 45}

In [28]:
''' this is working out how to hanlde the data and pull the mid slice from each of these images'''
img_filename_lst = ['stad425_rois002.nd2', 'stad425_rois004.nd2'] #list of .nd2 files to be read

'''iterate over each files of view in eahc .nds file and save the most in focus single z-slice into a new .tiff file'''

for img_file in :
    multipoints_nd2 = ND2Reader(img_file) #read in .nd2 file. this is either 5 or 6 dimensional pims like image
    multipoints_nd2.bundle_axes = 'czyx' #bundle the czyx dimensions together
    if 'v' in multipoints_nd2.sizes: # this checks to see if there are multipole fields of view in the .nd2 image. It will iterate over FOVs if this is the case
        multipoints_nd2.iter_axes = 'v' #turn the field of view int othe 'index' fro the the images
        multipoints_5d_array = np.array(multipoints_nd2).astype('uint16') #converst the image into a 16bit array for easier handling. It takes aa moment to read this large file into memory but I think that it is worht i int hte longs end
        
        for fov in multipoints_5d_array: # iterate over the fields of view
            dcts_lst = [] #list to be appended with the dcts of each slice in the image
            dapi_3d = multipoints_5d_array[fov, 3, :, :, :] #pull out just the dapi channel
            for zslice in range(dapi_3d.shape[0]): #iterate over the zslices in the dapi image
                dapi_slice = dapi_3d[zslice, :, :] #pull out an individaul slice
                dcts_lst.append(dcts(dapi_3d)) #append the shannaon entropy of discreate cosine transform to list
            min_entropy = np.argwhere(np.array(dcts_lst) == np.min(dcts_lst))  #find position in the list of the minimum entropy
            os.chdir(new_img_dir) #change to new directory for image
            for channel in range(img_5d_multi.shape[1]):
                io.imsave(filename[:-4])
    else: # if the .nd2 file only contains a single field of view than it heads down this path. This is the same as above minus an iteration
        dcts_lst = [] #empyt list to be poulated with DCTS of each z-slice in the image
        multipoints_4d_array = np.array(multipoints_nd2).astype('uint16') #convert .nd2 file into numpy arra
        dapi_3d = multipoints_4d_array[3, :, :, :] # pull out just the dapi chanel
        for zslice in range(dapi_3d.shape[0]): #iterate over the z-slices in the dapi image
                dapi_slice = dapi_3d[zslice, :, :] #look at a single slice in the dapi channel
                dcts_lst.append(dcts(dapi_slice)) #determine the DCTS of the image and add it to running list
     
            

In [40]:
img_filename_lst[0][:-4]

'stad425_rois002'

In [24]:
roi2_multipoints.bundle_axes = 'czyx'

In [25]:
roi4_multipoints.bundle_axes = 'czyx'

In [45]:
print(imgs)

<FramesSequenceND>
Axes: 6
Axis 'x' size: 2048
Axis 'y' size: 2044
Axis 'c' size: 4
Axis 't' size: 1
Axis 'z' size: 25
Axis 'v' size: 5
Pixel Datatype: <class 'numpy.float64'>
